<a href="https://colab.research.google.com/github/mustafabozkaya/Colab_uygulamalar/blob/main/Sentiment_Analysis_With_SparkNLP_and%C2%A0Comet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spark-nlp==4.2.6
!pip install comet_ml

In [ ]:
!pip install pyspark

In [ ]:
# Import Spark NLP
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

spark = sparknlp.start()

# Import comet
import comet_ml
from sparknlp.logging.comet import CometLogger

In [ ]:
experiment = comet_ml.Experiment("bJKpx3E8fy3xLnFRyoTUS13Yf")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/jamessandy/general/a7e641c215be4beca7cfb8de6f3a545a



In [ ]:
comet_ml.init(project_name='test-sparknlp ')

Please enter your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Comet API key is valid
COMET WARNING: running in Google Colab, but can't find mounted drive. Using HOME instead
COMET WARNING: if drive is mounted, set COMET_CONFIG to save config there
COMET INFO: Comet API key saved in /root/.comet.config


In [ ]:
spark = sparknlp.start()
logger = CometLogger()
OUTPUT_LOG_PATH = "./run"

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/jamessandy/general/a7e641c215be4beca7cfb8de6f3a545a
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git P

In [ ]:
# Create a document assembler
document = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

In [ ]:
# Create a tokenizer
token = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

In [ ]:
# Create a normalizer
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normal")

In [ ]:
# Load a pretrained sentiment analysis model
vivekn = ViveknSentimentModel.pretrained() \
    .setInputCols(["document", "normal"]) \
    .setOutputCol("result_sentiment")

sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
[OK!]


In [ ]:
# Create a finisher
finisher = Finisher() \
    .setInputCols(["result_sentiment"]) \
    .setOutputCols("final_sentiment")


In [ ]:
# Create a pipeline with all the stages
pipeline = Pipeline().setStages([document, token, normalizer, vivekn, finisher])

# Create the data as a Spark dataframe
data = spark.createDataFrame([
    ["I recommend this movie"],
    ["Dont waste your time!!!"]
]).toDF("text")

In [ ]:
# Fit the pipeline to the data
pipelineModel = pipeline.fit(data)

# Transform the data and show the final sentiment
result = pipelineModel.transform(data)
result.select("final_sentiment").show(truncate=False)
logger.log_pipeline_parameters(pipelineModel)
logger.end()

+---------------+
|final_sentiment|
+---------------+
|[positive]     |
|[positive]     |
+---------------+



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/jamessandy/test-sparknlp/d23eacfe5a394f2ca33a2cda891bc577
COMET INFO:   Others:
COMET INFO:     Created from : SparkNLP
COMET INFO:     model_uid    : VIVEKN_8ac4c5e76848
COMET INFO:     pipeline_uid : PipelineModel_98efb2e744f6
COMET INFO:   Parameters:
COMET INFO:     DocumentAssembler-cleanupMode               : disabled
COMET INFO:     DocumentAssembler-inputCol                  : text
COMET INFO:     DocumentAssembler-outputCol                 : document
COMET INFO:     Finisher-annotationSplitSymbol              : @
COMET INFO:     Finisher-cleanAnnotations                   : True
COMET INFO:     Finisher-includeMetadata                    : False
COMET INFO:     Finisher-inputCols                          : ['result_sentiment']
COMET INFO